In [1]:
import pandas as pd
import os

## Data Reading

In [2]:
df_mapping_no2_to_traffico = pd.read_csv("../../data/processed/mapping/no2_to_traffic_sensor_mapping.csv")

df_air_quality_and_locations = pd.read_parquet("../../data/processed/air/df_air_quality_and_locations_from_2013.parquet")

In [3]:
# Define the root directory for processed traffic data
root_dir = '../../data/processed/traffic'

# List to hold DataFrames
dataframes = []

# Iterate through all folders in the root directory
for folder_name in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder_name)
    
    # Check if the folder exists and is a directory
    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder_path}")
        
        # Iterate through all files in the folder
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.parquet'):  # Ensure it's a Parquet file
                file_path = os.path.join(folder_path, file_name)
                
                # Read the Parquet file
                try:
                    df = pd.read_parquet(file_path)
                    print(f"Successfully read: {file_path} with {len(df)} rows.")
                    
                    # Append the DataFrame to the list
                    dataframes.append(df)
                    
                except Exception as e:
                    print(f"Error reading file {file_name}: {e}")

# Concatenate all DataFrames into one
if dataframes:  # Check if the list is not empty
    df_traffic = pd.concat(dataframes, ignore_index=True)
    print(f"Combined DataFrame created with {len(df_traffic)} rows.")

Processing folder: ../../data/processed/traffic/2022
Successfully read: ../../data/processed/traffic/2022/12-2022_processed.parquet with 48331 rows.
Successfully read: ../../data/processed/traffic/2022/11-2022_processed.parquet with 46722 rows.
Successfully read: ../../data/processed/traffic/2022/10-2022_processed.parquet with 47576 rows.
Successfully read: ../../data/processed/traffic/2022/01-2022_processed.parquet with 45047 rows.
Successfully read: ../../data/processed/traffic/2022/06-2022_processed.parquet with 42196 rows.
Successfully read: ../../data/processed/traffic/2022/05-2022_processed.parquet with 42735 rows.
Successfully read: ../../data/processed/traffic/2022/08-2022_processed.parquet with 45087 rows.
Successfully read: ../../data/processed/traffic/2022/02-2022_processed.parquet with 40460 rows.
Successfully read: ../../data/processed/traffic/2022/03-2022_processed.parquet with 44372 rows.
Successfully read: ../../data/processed/traffic/2022/09-2022_processed.parquet with

In [4]:
df_traffic = df_traffic.rename(columns = {'hora': 'fecha'})

In [33]:
df_traffic.to_parquet("traffic_data.parquet", index = False)

In [35]:
df_traffic.fecha.min()

Timestamp('2018-01-01 00:00:00')

## Data processing for NO2 Sensores

1. Eliminar del dataset todos los sensores que no esten dentro del dataset "df_mapping_no2_to_traffico".
para ello, compara del dataset df_mapping_no2_to_traffico columna id_no2 con dataset df_air_quality_and_locations "CODIGO".

In [5]:
# Filter df_air_quality_and_locations to keep only sensors present in df_mapping_no2_to_traffico
filtered_air_quality_data = df_air_quality_and_locations[
    df_air_quality_and_locations['CODIGO'].isin(df_mapping_no2_to_traffico['id_no2'])
]

In [6]:
# Get the length of the original DataFrame
original_count = len(df_air_quality_and_locations)

# Get the length of the filtered DataFrame
filtered_count = len(filtered_air_quality_data)

# Calculate the percentage of retained sensors
percentage_retained = (filtered_count / original_count) * 100

print(f'Number of original sensors: {original_count}')
print(f'Number of retained sensors: {filtered_count}')
print(f'Percentage of retained data: {percentage_retained:.2f}%')

Number of original sensors: 2490936
Number of retained sensors: 1343640
Percentage of retained data: 53.94%


Let's remove unnecesary columns:

In [7]:
unnecesary_cols = ['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','PUNTO_MUESTREO','CODIGO_CORTO','LOCALIZACION','DIRECCION','COD_VIA','COD_TIPO']

In [8]:
df_air = filtered_air_quality_data.drop(columns = unnecesary_cols)
df_air.columns = df_air.columns.str.lower()
df_air = df_air.rename(columns = {'codigo':'id_no2'})

# Creating the main dataset

### Approach 1: Count the period for each `ID_trafffic sensors`

In [9]:
# Create a new dataframe with the following structure, for each unique traffic sensor (id_trafico)

# id_traffico, year, month, appears
# 3411, 2024, 11, true
# 3411, 2024, 12, false
df = df_traffic.copy()

# Convert fecha to datetime
df["fecha"] = pd.to_datetime(df["fecha"])

# Extract year and month
df["year"] = df["fecha"].dt.year
df["month"] = df["fecha"].dt.month

# Create the new DataFrame with unique combinations
result = df.groupby(["id_trafico", "year", "month"]).size().reset_index(name="count")

# Add 'appears' column (True if there is data for that month)
result["appears"] = result["count"] > 500

# Drop the count column as it's not needed
result = result.drop(columns=["count"])

result.sort_values(by=['year', 'month', 'id_trafico'])

result = result[result['appears'] == True]

In [10]:
df_summary = result.groupby('id_trafico').agg(
    start=('year', lambda x: f"{x.iloc[0]}-{df.loc[x.index[0], 'month']:02d}"),
    end=('year', lambda x: f"{x.iloc[-1]}-{df.loc[x.index[-1], 'month']:02d}")
).reset_index()

ensure that each sensors has continuous data, you can see the year and month, check the max, check the min and create a new df for each sensor and a new variable: is_continuous




In [11]:
#df_summary.to_csv("traffic_sensor_periods.csv", index = False)

In [12]:
result[result['id_trafico'] == '6119'].sort_values(by=['year', 'month']).drop(columns = ['appears'])

,id_trafico,year,month
4502,6119,2018,1
4503,6119,2018,2
4504,6119,2018,3
4505,6119,2018,4
4506,6119,2018,5
...,...,...,...
4581,6119,2024,8
4582,6119,2024,9
4583,6119,2024,10
4584,6119,2024,11


In [13]:
def check_continuity(group):
    group = group.sort_values(['year', 'month'])
    
    # Create a full range of expected periods
    min_date = f"{group['year'].min()}-{group['month'].min()}-01"
    max_date = f"{group['year'].max()}-{group['month'].max()}-01"
    expected_range = pd.date_range(start=min_date, end=max_date, freq='MS').to_period('M')
    
    # Get actual periods in the dataset
    actual_range = pd.to_datetime(group[['year', 'month']].assign(day=1)).dt.to_period('M')
    
    # Check if the sets are equal
    is_continuous = set(expected_range) == set(actual_range)
    
    return pd.Series({
        'start': group.iloc[0][['year', 'month']].astype(str).str.cat(sep='-'),
        'end': group.iloc[-1][['year', 'month']].astype(str).str.cat(sep='-'),
        'is_continuous': is_continuous
    })

# Apply function to each sensor
df_final = result.groupby('id_trafico').apply(check_continuity).reset_index()

/var/folders/kc/t6c9n8gj7mbblj0w7sy3ykwm0000gn/T/ipykernel_11174/898840416.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final = result.groupby('id_trafico').apply(check_continuity).reset_index()


In [14]:
df_traffic[(df_traffic['id_trafico'] == '6123') & 
                               (df_traffic['fecha'].dt.year == 2019) & 
                               (df_traffic['fecha'].dt.month == 3)]

,id_trafico,fecha,intensidad,carga,ocupacion,vmed
1894673,6123,2019-03-01 00:00:00,79.75,4.341693,0.313480,0.0
1894674,6123,2019-03-01 01:00:00,39.00,2.903846,0.544872,0.0
1894675,6123,2019-03-01 02:00:00,25.75,1.757282,0.000000,0.0
1894676,6123,2019-03-01 03:00:00,24.00,0.802083,0.000000,0.0
1894677,6123,2019-03-01 04:00:00,12.00,0.541667,0.000000,0.0
...,...,...,...,...,...,...
1894757,6123,2019-03-04 12:00:00,0.00,0.000000,0.000000,0.0
1894758,6123,2019-03-04 13:00:00,120.00,14.000000,26.000000,0.0
1894759,6123,2019-03-12 08:00:00,0.00,0.000000,0.000000,0.0
1894760,6123,2019-03-13 16:00:00,0.00,0.000000,0.000000,0.0


In [15]:
#result[result['id_trafico'] == '6123'].sort_values(by=['year', 'month']).to_csv("test.csv", index = False)

In [16]:
#a = pd.read_csv('../../data/raw/traffic/2019/03-2019.csv',delimiter= ";")

In [17]:
#a[a['id'] == 6123].sort_values(by='fecha')

`Solo los sensores 5547, 5783, 5465, 5414, 5084, 4555, 4129, 3915, 3911, van del 2018-01 al 2024-12`

In [21]:
result.id_trafico.unique()

array(['10250', '10332', '10580', '10613', '10614', '10615', '10647',
       '10814', '10885', '10889', '10890', '10982', '11006', '11007',
       '3411', '3513', '3730', '3731', '3732', '3791', '3910', '3911',
       '3913', '3914', '3915', '3916', '3917', '4022', '4026', '4027',
       '4028', '4048', '4129', '4283', '4284', '4285', '4286', '4301',
       '4303', '4313', '4333', '4353', '4437', '4461', '4469', '4472',
       '4555', '5084', '5090', '5091', '5414', '5415', '5416', '5421',
       '5422', '5437', '5465', '5515', '5547', '5783', '5784', '5939',
       '6116', '6118', '6119', '6123', '6822', '6823', '7071'],
      dtype=object)

In [22]:
df_final

,id_trafico,start,end,is_continuous
0,10250,2018-12,2024-9,False
1,10332,2018-6,2018-8,True
2,10580,2020-7,2024-9,False
3,10613,2020-8,2024-12,False
4,10614,2020-8,2024-12,False
...,...,...,...,...
64,6119,2018-1,2024-12,False
65,6123,2018-1,2024-12,False
66,6822,2018-1,2024-12,False
67,6823,2018-1,2024-12,False


In [25]:
result.drop(columns = 'appears')

,id_trafico,year,month
1,10250,2018,12
2,10250,2019,1
3,10250,2019,2
4,10250,2019,3
5,10250,2019,4
...,...,...,...
4905,7071,2024,8
4906,7071,2024,9
4907,7071,2024,10
4908,7071,2024,11


In [30]:
df_traffic[df_traffic['id_trafico'] == '4303'].drop(columns = ['intensidad','carga','ocupacion','vmed'])

,id_trafico,fecha
26768,4303,2022-12-01 00:00:00
26769,4303,2022-12-01 01:00:00
26770,4303,2022-12-01 02:00:00
26771,4303,2022-12-01 03:00:00
26772,4303,2022-12-01 04:00:00
...,...,...
3418526,4303,2018-10-24 05:00:00
3418527,4303,2018-10-24 06:00:00
3418528,4303,2018-10-24 07:00:00
3418529,4303,2018-10-24 08:00:00
